<a href="https://colab.research.google.com/github/BaharehAm/-Bankruptcy-prediction-using-optimal-ensemble-models-under-balanced-and-imbalanced-data/blob/main/Hyperparameter_tuning_%2B_Wilcoxon_signed_rank_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
year = 'year2'

## Import libraries

In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
 pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from google.colab import output
import os


from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTEN

# from imblearn.over_sampling import ADASYN
# from imblearn.over_sampling import KMeansSMOTE
# from imblearn.over_sampling import SVMSMOTE

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
# from keras.layers import Input, Dense, SimpleRNN, LSTM, GRU, Bidirectional, Conv1D, MaxPooling1D, Flatten, concatenate, Dropout


import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [ ]:
import random
def randint(min=0,max=100):
    a = random.randint(min,max)
    return a

## Load data

In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/Research/BPY/Data/' + year + '.csv')
print(len(data1))
data1

10173


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,6,7,9,10,11,14,18,22,24,25,29,35,36,38,48,51,55,57,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class
0,1,0.20235,0.465,0.24038,1.5171,-14.547,0.51069,0.25366,0.91816,1.1519,...,0.13184,0.47395,0.86816,0.00023951,8.5487,5.1655,107.74,3.3879,5.344,0
1,2,0.030073,0.59563,0.18668,1.3382,-37.859,-0.00031864,0.04167,0.6789,0.32356,...,0.12146,0.074369,0.87235,0,1.5264,0.63305,622.66,0.58619,1.2381,0
2,3,0.25786,0.29949,0.66519,3.2211,71.799,0,0.31877,2.332,1.6762,...,0.16499,0.36921,0.81614,0,4.3325,3.1985,65.215,5.5969,47.466,0
3,4,0.22716,0.6785,0.042784,1.0828,-88.212,0,0.28505,0.47384,1.3241,...,0.29358,0.70657,0.78617,0.48456,5.2309,5.0675,142.46,2.5621,3.0066,0
4,5,0.085443,0.38039,0.35923,1.9444,21.731,0.1879,0.10823,1.3714,1.1126,...,0.10124,0.16379,0.89876,0,5.7035,4.002,89.058,4.0984,5.9874,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10168,10169,0.02997,0.66806,0.066243,1.1103,-105.55,0.02997,0.038888,0.48274,1.0292,...,0.028377,0.092931,0.97162,0.20982,3.0914,3.9456,192.22,1.8988,3.4199,1
10169,10170,0.012843,0.49306,-0.16062,0.61898,-24.801,0.012843,0.012843,0.9059,1.0145,...,0.014247,0.028752,0.98575,0.16009,48.666,63.752,40.071,9.1087,5.1956,1
10170,10171,0.015092,0.55759,-0.2846,0.48599,-85.571,0.015092,0.009826,0.69488,1.006,...,0.0059714,0.03895,0.99403,0.010091,15.053,11.964,114.25,3.1948,2.4201,1
10171,10172,-0.002554,0.47076,0.42401,1.9007,0.95483,-0.002554,0.001785,1.1144,0.99293,...,-0.0071218,-0.004869,1.0071,0.0,6.4289,5.7025,64.291,5.6773,25.399,1


## Handling missing data

In [ ]:
data1.iloc[1:2, 23:28]

,Attr23,Attr24,Attr25,Attr26,Attr27
1,0.092943,?,0.029802,0.059639,79.752


In [ ]:
data1_ = data1.iloc[:, 1:65]
data1_.columns
for column_name in data1_.columns:
  data1_[[column_name]]=data1_[[column_name]].replace("?",np.NaN).astype('float32')
data1_.describe()


,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr55,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64
count,10172.000000,10172.000000,10172.000000,10151.000000,10149.000000,10172.000000,10172.000000,10155.000000,10169.000000,10172.000000,...,1.017200e+04,10110.000000,10171.000000,10134.000000,10171.000000,9630.000000,10157.000000,1.011000e+04,10151.000000,9961.000000
mean,0.043074,0.646960,0.070861,4.144058,-144.830811,-0.111564,0.122101,9.868797,2.867541,0.344341,...,6.080925e+03,-1.128949,-0.043519,12.656337,3.034738,375.332489,16.884022,8.910800e+02,11.217654,55.275875
std,1.112028,6.615405,6.606494,51.715244,7811.976074,6.568088,6.532490,288.386871,96.674599,6.655844,...,6.598007e+04,86.502914,12.251651,632.937378,238.185654,12294.316406,295.762299,4.366134e+04,233.919220,1526.588379
min,-75.331001,0.000000,-479.959991,0.002079,-438250.000000,-508.410004,-75.331001,-1.594500,-0.612590,-479.910004,...,-1.805200e+06,-8534.599609,-979.250000,-4.549700,-189.580002,0.000000,-0.007521,-9.921400e+02,-0.367890,-10677.000000
25%,0.000364,0.277695,0.012114,1.029200,-50.131001,0.000000,0.001975,0.393635,1.026300,0.277365,...,5.699975e+00,0.010380,0.009959,0.872130,0.000000,5.684275,4.695300,4.028225e+01,3.102200,2.310900
50%,0.049493,0.487335,0.189695,1.524400,-1.907500,0.000000,0.060057,1.005400,1.285700,0.490925,...,8.326700e+02,0.055014,0.126210,0.949715,0.004849,10.353500,6.988600,6.964550e+01,5.198900,4.536000
75%,0.141105,0.705040,0.400400,2.741100,51.443001,0.074739,0.165865,2.518850,2.256600,0.703280,...,4.020250e+03,0.134067,0.309135,0.993390,0.249240,22.163500,11.073000,1.163200e+02,9.012550,10.604000
max,7.372700,480.959991,5.502200,4881.600098,70686.000000,35.550999,649.229980,18554.000000,9742.299805,74.433998,...,3.657400e+06,20.200001,147.190002,59672.000000,23853.000000,639940.000000,26862.000000,4.144800e+06,23454.000000,127680.000000


In [ ]:
data1_.iloc[1:2, 23:28]

,Attr24,Attr25,Attr26,Attr27,Attr28
1,NaN,0.029802,0.059639,79.751999,0.71432


In [ ]:
for column_name in data1_.columns:
  data1_[column_name].fillna(data1_[column_name].mean(), inplace=True)
  # data1_[column_name].fillna(0, inplace=True)
data1_.describe()

,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr55,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64
count,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,...,1.017300e+04,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,10173.000000,1.017300e+04,10173.000000,10173.000000
mean,0.043074,0.646960,0.070861,4.144058,-144.830795,-0.111564,0.122101,9.868797,2.867541,0.344341,...,6.080925e+03,-1.128949,-0.043519,12.656336,3.034738,375.332520,16.884020,8.910799e+02,11.217653,55.275879
std,1.111974,6.615080,6.606170,51.659290,7802.754883,6.567765,6.532169,288.131592,96.655586,6.655517,...,6.597683e+04,86.234619,12.250446,631.722900,238.162231,11961.669922,295.529602,4.352592e+04,233.666122,1510.596436
min,-75.331001,0.000000,-479.959991,0.002079,-438250.000000,-508.410004,-75.331001,-1.594500,-0.612590,-479.910004,...,-1.805200e+06,-8534.599609,-979.250000,-4.549700,-189.580002,0.000000,-0.007521,-9.921400e+02,-0.367890,-10677.000000
25%,0.000368,0.277710,0.012133,1.030100,-50.783001,0.000000,0.001985,0.393950,1.026300,0.277510,...,5.720000e+00,0.009550,0.009951,0.872580,0.000000,5.907300,4.698600,4.043400e+01,3.105600,2.342400
50%,0.049464,0.487340,0.189660,1.527400,-2.116600,0.000000,0.060060,1.007500,1.285900,0.490920,...,8.331200e+02,0.054562,0.126160,0.950280,0.004866,11.032000,6.997200,7.015800e+01,5.212600,4.657300
75%,0.141100,0.705030,0.400350,2.758600,51.243000,0.074718,0.165850,2.539700,2.259300,0.703270,...,4.021600e+03,0.133340,0.309130,0.993810,0.249410,27.108000,11.112000,1.177200e+02,9.039500,11.359000
max,7.372700,480.959991,5.502200,4881.600098,70686.000000,35.550999,649.229980,18554.000000,9742.299805,74.433998,...,3.657400e+06,20.200001,147.190002,59672.000000,23853.000000,639940.000000,26862.000000,4.144800e+06,23454.000000,127680.000000


In [ ]:
data1_.iloc[1:2, 23:28]

,Attr24,Attr25,Attr26,Attr27,Attr28
1,0.240073,0.029802,0.059639,79.751999,0.71432


## Statistics

In [ ]:

# years = ['year1', 'year2','year3','year4','year5']

# statistics = []
# for year in years:
#   data = pd.read_csv('/content/drive/MyDrive/Research/BPY/Data/' + year + '.csv')
#   statistics.append([year, len(data), round(len(data[data['class']==1])/len(data), 4)*100, round(len(data[data['class']==0])/len(data), 4)*100 ])

# statistics = pd.DataFrame(statistics, columns=['Year', 'N', '% Class 1', '% Class 0'])

# statistics

## Oversampling & Matrix building (new)

In [ ]:
def Over_sample(Over_sample_method, random_state, ratio, X_train, y_train):
  if ratio != 0:
    count_0 =  len(y_train)-y_train.sum()
    count_1_res = int(count_0 * ratio)
    if Over_sample_method == 'SMOTE':
      oversampler = SMOTE(random_state = random_state, sampling_strategy = {1:count_1_res})
    elif Over_sample_method == 'BorderlineSMOTE':
      oversampler = BorderlineSMOTE(random_state = random_state, sampling_strategy = {1:count_1_res})
    else:
      oversampler = SMOTEN(random_state = random_state, sampling_strategy = {1:count_1_res})
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    put_away = (len(y_train_res)-len(y_train)) / len(y_train_res)
    X_train_res_red, _ ,y_train_res_red, _ = train_test_split(X_train_res, y_train_res, test_size=put_away, random_state=random_state, stratify = y_train_res)
    # count_0_res_red = len(y_train_res_red[y_train_res_red['class']==0])
    # count_1_res_red = len(y_train_res_red[y_train_res_red['class']==1])
    # ratio_res_red = count_1_res_red / count_0_res_red
  else:
    X_train_res_red = X_train
    y_train_res_red = y_train
  return X_train_res_red, y_train_res_red

In [ ]:
# def Create_image_label(Over_sample_method, random_state, ratio, X_train, y_train, X_test, y_test):

#   X_train = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)[0]
#   y_train = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)[1]

#   all_mat = []
#   for i in range(len(X_train)):
#     mat = []
#     for j in range(8):
#       mat.append(X_train[i][j*8:(j+1)*8])
#     all_mat.append(mat)
#   images = np.array(all_mat)
#   images_train = images.reshape((images.shape[0], images.shape[1], images.shape[2], 1))
#   labels_train = y_train.reshape((y_train.shape[0], 1))

#   all_mat_test = []
#   for i in range(len(X_test)):
#     mat = []
#     for j in range(8):
#       mat.append(X_test[i][j*8:(j+1)*8])
#     all_mat_test.append(mat)
#   images = np.array(all_mat_test)
#   images_test = images.reshape((images.shape[0], images.shape[1], images.shape[2], 1))
#   labels_test = y_test.reshape((y_test.shape[0], 1))

#   return images_train, labels_train, images_test, labels_test


## K-fold CV

In [ ]:
 ##  cv[i][j]  -->   i: 0,1,...,k (number of folds)    j--> 0:X_train,  1:y_train,  2:X_test, 3: y_test
X_orig = np.array(data1_)
y_orig = np.array(data1.iloc[:, -1:])
np.shape(X_orig), np.shape(y_orig)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
cv = {}
indexes = {}
for i, (train_index, test_index) in enumerate(skf.split(X_orig, y_orig)):
    X_train, X_test = X_orig[train_index], X_orig[test_index]
    y_train, y_test = y_orig[train_index], y_orig[test_index]
    indexes[i]=[train_index, test_index]
    cv[i]=[X_train, y_train, X_test, y_test]

print("the ratio of class '1' to class '0' in total, training and testing sets are: %f, %f, and %f" %(
                                                                                                      y_orig.sum()/(len(y_orig)-y_orig.sum()),
                                                                                                      y_train.sum()/(len(y_train)-y_train.sum()),
                                                                                                      y_test.sum()/(len(y_test)-y_test.sum())
                                                                                                      ))

the ratio of class '1' to class '0' in total, training and testing sets are: 0.040929, 0.040928, and 0.040942


In [ ]:
## Hyper-parameter tuning of CTB:

model_name = "CTB"
random_state = randint()
print(year, model_name, random_state)

Over_sample_method = 'SMOTE'
ratio = 0


depths = [3, 7, 12]
learning_rates = [0.3, 0.1, 0.01]
iterations_set = [100, 500]
l2_leaf_regs = [1, 10]
model_size_regs = [1, 10]
od_wait = 0.01
early_stopping_rounds = 50

tuning_results_CTB = []
for depth in depths:
  for learning_rate in learning_rates:
    for iterations in iterations_set:
      for l2_leaf_reg in l2_leaf_regs:
        for model_size_reg in model_size_regs:

          model = CatBoostClassifier(iterations=iterations, learning_rate=learning_rate, depth=depth, od_wait=od_wait,
                                     l2_leaf_reg=l2_leaf_reg, model_size_reg=model_size_reg, loss_function='Logloss')
          AUCs_CTB = []
          ACCs_CTB = []
          for key, value in cv.items():
            X_train = value[0]
            y_train = value[1]
            X_test = value[2]
            y_test = value[3]

            X_y_over_sampled = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)
            X = X_y_over_sampled[0]
            y = X_y_over_sampled[1]
            X_t = X_test
            y_t = y_test

            eval_set = (X_t, y_t)
            eval_metric = ["auc", "error", "logloss"]
            history = model.fit(X, y, eval_set=eval_set, early_stopping_rounds=early_stopping_rounds, use_best_model=True, verbose=False)
            ACC = model.score(X_t, y_t)
            y_pred = model.predict(X_t)
            prob_pred = model.predict_proba(X_t)
            AUC = metrics.roc_auc_score(y_t, prob_pred[:,1])
            AUCs_CTB.append(AUC)
            ACCs_CTB.append(ACC)

          tuning_result = [random_state, model_name, Over_sample_method, ratio,
                           depth, learning_rate, iterations, l2_leaf_reg, model_size_reg,
                           np.mean(ACCs_CTB), np.std(ACCs_CTB), np.mean(AUCs_CTB), np.std(AUCs_CTB)]
          print(tuning_result)
          tuning_results_CTB.append(tuning_result)

columns = ['random_state', 'model_name', 'Over_sample_method', 'ratio',
                           'depth', 'learning_rate', 'iterations', 'l2_leaf_reg', 'model_size_reg',
                           'AVG_ACC', 'STD_ACC', 'AVG_AUC', 'STD_AUC']
tuning_results_CTB = pd.DataFrame(tuning_results_CTB, columns =columns).sort_values(by=['AVG_AUC'], ascending=False)

os.chdir('/content/drive/MyDrive/Research/BPY/Tuning_Results')
tuning_results_CTB.to_csv(year + '_tuning_results_' + model_name + '_' + Over_sample_method + '_' + str(ratio) +'.csv')
tuning_results_CTB

year2 CTB 35
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 100, 1, 1, 0.9443775211999235, 0.003141997316169087, 0.9134605329532205, 0.01476456719731865]
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 100, 1, 10, 0.9443775211999235, 0.003141997316169087, 0.9134605329532205, 0.01476456719731865]
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 100, 10, 1, 0.9447400585417988, 0.0025997981327849006, 0.915839853438911, 0.021580764847789414]
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 100, 10, 10, 0.9447400585417988, 0.0025997981327849006, 0.915839853438911, 0.021580764847789414]
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 500, 1, 1, 0.9441062287027323, 0.003528779005623598, 0.9186507515389751, 0.018430023222731216]
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 500, 1, 10, 0.9441062287027323, 0.003528779005623598, 0.9186507515389751, 0.018430023222731216]
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 500, 10, 1, 0.9439252042354808, 0.002807436925411026, 0.9216323733184302, 0.024425953306282906]
[35, 'CTB', 'SMOTE', 0, 3, 0.3, 500, 10, 10, 0.9439252042354808, 0.002807436925411026, 0.92163

,random_state,model_name,Over_sample_method,ratio,depth,learning_rate,iterations,l2_leaf_reg,model_size_reg,AVG_ACC,STD_ACC,AVG_AUC,STD_AUC
39,35,CTB,SMOTE,0,7,0.10,500,10,10,0.944469,0.002298,0.937879,0.015651
38,35,CTB,SMOTE,0,7,0.10,500,10,1,0.944469,0.002298,0.937879,0.015651
31,35,CTB,SMOTE,0,7,0.30,500,10,10,0.943653,0.001652,0.933521,0.018385
30,35,CTB,SMOTE,0,7,0.30,500,10,1,0.943653,0.001652,0.933521,0.018385
36,35,CTB,SMOTE,0,7,0.10,500,1,1,0.944740,0.001955,0.933164,0.009592
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,35,CTB,SMOTE,0,12,0.01,100,10,10,0.953706,0.002777,0.860257,0.022594
18,35,CTB,SMOTE,0,3,0.01,100,10,1,0.960680,0.000018,0.847127,0.013170
19,35,CTB,SMOTE,0,3,0.01,100,10,10,0.960680,0.000018,0.847127,0.013170
16,35,CTB,SMOTE,0,3,0.01,100,1,1,0.960680,0.000018,0.845500,0.010893


In [ ]:
## Hyper-parameter tuning of LGBM:

model_name = "LGBM"
random_state = randint()
print(year, model_name, random_state)

Over_sample_method = 'SMOTE'
ratio = 0.5



max_depths = [3, 7, 12]
learning_rates = [0.3, 0.1, 0.01]
n_estimators_set = [100, 500]
reg_lambdas = [1, 10]
reg_alphas = [1, 10]
min_split_gains = [0, 0.01, 0.1]
early_stopping_rounds = 50

tuning_results_LGBM = []
for max_depth in max_depths:
  for learning_rate in learning_rates:
    for n_estimators in n_estimators_set:
      for reg_lambda in reg_lambdas:
        for reg_alpha in reg_alphas:
          for min_split_gain in min_split_gains:

            model = LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, num_leaves=2**max_depth,
                                  reg_lambda=reg_lambda, reg_alpha=reg_alpha, min_split_gain=min_split_gain)
            AUCs_LGBM = []
            ACCs_LGBM = []
            for key, value in cv.items():
              X_train = value[0]
              y_train = value[1]
              X_test = value[2]
              y_test = value[3]

              X_y_over_sampled = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)
              X = X_y_over_sampled[0]
              y = np.ravel(X_y_over_sampled[1])
              X_t = X_test
              y_t = np.ravel(y_test)

              eval_set = [(X, y), (X_t, y_t)]
              eval_metric = ["auc", "error", "logloss"]
              history = model.fit(X, y, early_stopping_rounds=early_stopping_rounds, eval_set=eval_set, eval_metric=eval_metric, verbose=False)
              ACC = model.score(X_t, y_t)
              y_pred = model.predict(X_t)
              prob_pred = model.predict_proba(X_t)
              AUC = metrics.roc_auc_score(y_t, prob_pred[:,1])
              AUCs_LGBM.append(AUC)
              ACCs_LGBM.append(ACC)

            tuning_result = [random_state, model_name, Over_sample_method, ratio,
                            learning_rate, n_estimators, max_depth, reg_lambda, reg_alpha, min_split_gain,
                            np.mean(ACCs_LGBM), np.std(ACCs_LGBM), np.mean(AUCs_LGBM), np.std(AUCs_LGBM)]
            print(tuning_result)
            tuning_results_LGBM.append(tuning_result)

columns = ['random_state', 'model_name', 'Over_sample_method', 'ratio',
                           'learning_rate', 'n_estimators', 'max_depth', 'reg_lambda', 'reg_alpha', 'min_split_gain',
                           'AVG_ACC', 'STD_ACC', 'AVG_AUC', 'STD_AUC']
tuning_results_LGBM = pd.DataFrame(tuning_results_LGBM, columns =columns).sort_values(by=['AVG_AUC'], ascending=False)

os.chdir('/content/drive/MyDrive/Research/BPY/Tuning_Results')
tuning_results_LGBM.to_csv(year + '_tuning_results_' + model_name + '_' + Over_sample_method + '_' + str(ratio) +'.csv')
tuning_results_LGBM

In [ ]:
## Hyper-parameter tuning of XGB:

model_name = "XGB"
random_state = randint()
print(year, model_name, random_state)

Over_sample_method = 'SMOTE'
ratio = 0

max_depths = [5, 10]
learning_rates = [0.3, 0.1, 0.01]
n_estimators_set = [100, 500]
gammas = [1, 10]
reg_lambdas = [1, 10]
reg_alphas = [1, 10]
min_child_weights =[1, 10]
early_stopping_rounds = 50

tuning_results_XGB = []
for max_depth in max_depths:
  for learning_rate in learning_rates:
    for n_estimators in n_estimators_set:
      for gamma in gammas:
        for reg_lambda in reg_lambdas:
          for reg_alpha in reg_alphas:
            for min_child_weight in min_child_weights:

              model = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, gamma=gamma,
                                        reg_lambda=reg_lambda, reg_alpha=reg_alpha, min_child_weight=min_child_weight)
              AUCs_XGB = []
              ACCs_XGB = []
              for key, value in cv.items():
                X_train = value[0]
                y_train = value[1]
                X_test = value[2]
                y_test = value[3]

                X_y_over_sampled = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)
                X = X_y_over_sampled[0]
                y = np.ravel(X_y_over_sampled[1])
                X_t = X_test
                y_t = np.ravel(y_test)

                eval_set = [(X, y), (X_t, y_t)]
                eval_metric = ["auc", "error", "logloss"]
                history = model.fit(X, y, early_stopping_rounds=early_stopping_rounds, eval_set=eval_set, eval_metric=eval_metric, verbose=False)
                ACC = model.score(X_t, y_t)
                y_pred = model.predict(X_t)
                prob_pred = model.predict_proba(X_t)
                AUC = metrics.roc_auc_score(y_t, prob_pred[:,1])
                AUCs_XGB.append(AUC)
                ACCs_XGB.append(ACC)

              tuning_result = [random_state, model_name, Over_sample_method, ratio,
                              n_estimators, max_depth, learning_rate, gamma, reg_lambda, reg_alpha, min_child_weight,
                              np.mean(ACCs_XGB), np.std(ACCs_XGB), np.mean(AUCs_XGB), np.std(AUCs_XGB)]
              print(tuning_result)
              tuning_results_XGB.append(tuning_result)

columns = ['random_state', 'model_name', 'Over_sample_method', 'ratio',
                           'n_estimators', 'max_depth', 'learning_rate', 'gamma', 'reg_lambda', 'reg_alpha', 'min_child_weight',
                           'AVG_ACC', 'STD_ACC', 'AVG_AUC', 'STD_AUC']
tuning_results_XGB = pd.DataFrame(tuning_results_XGB, columns =columns).sort_values(by=['AVG_AUC'], ascending=False)

os.chdir('/content/drive/MyDrive/Research/BPY/Tuning_Results')
tuning_results_XGB.to_csv(year + '_tuning_results_' + model_name + '_' + Over_sample_method + '_' + str(ratio) +'.csv')
tuning_results_XGB

## Ensemble models

In [ ]:
## Load best Hyper-parameters for each model
os.chdir('/content/drive/MyDrive/Research/BPY/Tuning_Results')

Over_sample_method = 'SMOTE'
ratio = 0.5
early_stopping_rounds = 50
od_wait = 0.01
use_best_model=True


model_name = 'Catboost'
params = pd.read_csv(year + '_tuning_results_' + model_name + '_' + Over_sample_method + '_' + str(ratio) +'.csv').loc[0]
i3 = params.avg_AUC
depth = params.depth
learning_rate0 = params.learning_rate
l2_leaf_reg = params.l2_leaf_reg
model_size_reg = params.model_size_reg
iterations = params.iterations


model_name = 'LGBM'
params = pd.read_csv(year + '_tuning_results_' + model_name + '_' + Over_sample_method + '_' + str(ratio) +'.csv').loc[0]
j3 = params.avg_AUC
max_depth = params.max_depth
learning_rate = params.learning_rate
reg_lambda = params.reg_lambda
reg_alpha = params.reg_alpha
min_split_gain = params.min_split_gain
n_estimators = params.n_estimators

model_name = 'XGB'
params = pd.read_csv(year + '_tuning_results_' + model_name + '_' + Over_sample_method + '_' + str(ratio) +'.csv').loc[0]
k3 = params.avg_AUC
max_depth2 = params.max_depth
learning_rate2 = params.learning_rate
gamma = params.gamma
reg_lambda2 = params.reg_lambda
reg_alpha2 = params.reg_alpha
min_child_weight =  params.min_child_weight
n_estimators = params.n_estimators


In [ ]:
### 10-fold CV (Correct)
random_state = randint()
print('random_state: ' , random_state)

X_orig = np.array(data1_)
y_orig = np.array(data1.iloc[:, -1:])
np.shape(X_orig), np.shape(y_orig)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
cv = {}
indexes = {}
for i, (train_index, test_index) in enumerate(skf.split(X_orig, y_orig)):
    X_train, X_test = X_orig[train_index], X_orig[test_index]
    y_train, y_test = y_orig[train_index], y_orig[test_index]
    indexes[i]=[train_index, test_index]
    cv[i]=[X_train, y_train, X_test, y_test]
print("the ratio of class '1' to class '0' in total, training and testing sets are: %f, %f, and %f"
      %( y_orig.sum()/(len(y_orig)-y_orig.sum()), y_train.sum()/(len(y_train)-y_train.sum()), y_test.sum()/(len(y_test)-y_test.sum())))

AUCs_CTB = []
AUCs_LGBM = []
AUCs_XGB = []
AUC1s = []
AUC2s = []
AUC3s = []
AUC4s = []

ACCs_CTB= []
ACCs_LGBM = []
ACCs_XGB = []
ACC1s = []
ACC2s = []
ACC3s = []

for key, value in cv.items():
  X_train = value[0]
  y_train = value[1]
  X_test = value[2]
  y_test = value[3]

  preds = []

  AUC_ = []

  X_y_over_sampled = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)
  X = X_y_over_sampled[0]
  y = X_y_over_sampled[1]
  X_t = X_test
  y_t = y_test
  model = CatBoostClassifier(iterations=iterations, learning_rate=learning_rate0, depth=depth, od_wait=od_wait, l2_leaf_reg=l2_leaf_reg, model_size_reg=model_size_reg, loss_function='Logloss' )
  eval_set = (X_t, y_t)
  eval_metric = ["auc", "error", "logloss"]
  history_CTB = model.fit(X, y, eval_set=eval_set, early_stopping_rounds=early_stopping_rounds, use_best_model=True, verbose=False)
  ACC = model.score(X_t, y_t)
  y_pred = model.predict(X_t)
  preds.append(y_pred)
  prob_pred = model.predict_proba(X_t)
  AUC = metrics.roc_auc_score(y_t, prob_pred[:,1])
  print('Fold %.f, CTB AUC: %.5F' %(key+1, AUC))
  prob_pred_CTB = pd.DataFrame(prob_pred)
  prob_pred_CTB.columns=['class0-CTB', 'class1-CTB']
  prob_pred_CTB[['class1-CTB']]
  AUCs_CTB.append(AUC)
  ACCs_CTB.append(ACC)
  AUC_.append(AUC)



  X_y_over_sampled = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)
  X = X_y_over_sampled[0]
  y = np.ravel(X_y_over_sampled[1])
  X_t = X_test
  y_t = np.ravel(y_test)
  model = LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, num_leaves=2**max_depth,
                          reg_lambda=reg_lambda, reg_alpha=reg_alpha, min_split_gain=min_split_gain)
  eval_set = [(X, y), (X_t, y_t)]
  eval_metric = ["auc", "error", "logloss"]
  history_lgbm = model.fit(X, y, early_stopping_rounds=early_stopping_rounds, eval_set=eval_set, eval_metric=eval_metric, verbose=False)
  ACC = model.score(X_t, y_t)
  y_pred = model.predict(X_t)
  preds.append(y_pred)
  prob_pred = model.predict_proba(X_t)
  AUC = metrics.roc_auc_score(y_t, prob_pred[:,1])
  print('Fold %.f, LGBM AUC: %.5F' %(key+1, AUC))
  prob_pred_lgbm = pd.DataFrame(prob_pred)
  prob_pred_lgbm.columns=['class0-LGBM', 'class1-LGBM']
  prob_pred_lgbm[['class1-LGBM']]
  AUCs_LGBM.append(AUC)
  ACCs_LGBM.append(ACC)
  AUC_.append(AUC)



  X_y_over_sampled = Over_sample(Over_sample_method, random_state, ratio, X_train, y_train)
  X = X_y_over_sampled[0]
  y = np.ravel(X_y_over_sampled[1])
  X_t = X_test
  y_t = np.ravel(y_test)
  model = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=max_depth2, learning_rate=learning_rate2, gamma=gamma,
                            reg_lambda=reg_lambda2, reg_alpha=reg_alpha2, min_child_weight=min_child_weight)
  eval_set = [(X, y), (X_t, y_t)]
  eval_metric = ["auc", "error", "logloss"]
  history_xgb = model.fit(X, y, early_stopping_rounds=early_stopping_rounds, eval_set=eval_set, eval_metric=eval_metric, verbose=False)
  ACC = model.score(X_t, y_t)
  y_pred = model.predict(X_t)
  preds.append(y_pred)
  prob_pred = model.predict_proba(X_t)
  AUC = metrics.roc_auc_score(y_t, prob_pred[:,1])
  print('Fold %.f, XGB AUC: %.5F' %(key+1, AUC))
  prob_pred_xgb = pd.DataFrame(prob_pred)
  prob_pred_xgb.columns=['class0-XGB', 'class1-XGB']
  prob_pred_xgb[['class1-XGB']]
  AUCs_XGB.append(AUC)
  ACCs_XGB.append(ACC)
  AUC_.append(AUC)



  ## Soft voting
  probs = pd.concat([prob_pred_CTB, prob_pred_lgbm, prob_pred_xgb], axis=1)
  probs.columns = ['0_ctb', '1_ctb', '0_LGBM', '1_LGBM', '0_XGB', '1_XGB']
  probs['mean_0'] = probs[['0_ctb', '0_LGBM', '0_XGB']].mean(axis=1)
  probs['mean_1'] = probs[['1_ctb', '1_LGBM', '1_XGB']].mean(axis=1)
  AUC1 = metrics.roc_auc_score(y_t, probs['mean_1'])
  print('Fold %.f, Ensemble1 AUC: %.5F' %(key+1, AUC1))
  AUC1s.append(AUC1)
  # y_pred = (prob_pred > 0.5).astype("int32")


  ## Weighted Soft voting 1
  i = 0.8 if np.argmax(AUC_) == 0 else 0.1
  j = 0.8 if np.argmax(AUC_) == 1 else 0.1
  k = 0.8 if np.argmax(AUC_) == 2 else 0.1
  weights1 = [i, j, k]
  probs['W_mean_0'] = np.average(probs[['0_ctb', '0_LGBM', '0_XGB']], weights=weights1, axis=1)
  probs['W_mean_1'] = np.average(probs[['1_ctb', '1_LGBM', '1_XGB']], weights=weights1, axis=1)
  AUC2 = metrics.roc_auc_score(y_t, probs['W_mean_1'])
  print('Fold %.f, Ensemble2 AUC: %.5F' %(key+1, AUC2))
  print('weights: ' , weights1)
  AUC2s.append(AUC2)

  ## Weighted Soft voting 2
  i2 = 0.5 if np.argmax(AUC_) == 0 else 0.25
  j2 = 0.5 if np.argmax(AUC_) == 1 else 0.25
  k2 = 0.5 if np.argmax(AUC_) == 2 else 0.25
  weights2 = [i2, j2, k2]
  probs['W_mean_0'] = np.average(probs[['0_ctb', '0_LGBM', '0_XGB']], weights=weights2, axis=1)
  probs['W_mean_1'] = np.average(probs[['1_ctb', '1_LGBM', '1_XGB']], weights=weights2, axis=1)
  AUC3 = metrics.roc_auc_score(y_t, probs['W_mean_1'])
  print('Fold %.f, Ensemble3 AUC: %.5F' %(key+1, AUC3))
  print('weights: ' , weights2)
  AUC3s.append(AUC3)

  ## Ensemble 4
  probs = pd.concat([prob_pred_CTB[['class1-CTB']], prob_pred_lgbm[['class1-LGBM']], prob_pred_xgb[['class1-XGB']] ] , axis=1)
  probs_sorted = pd.DataFrame(np.sort(probs.values, axis=1)[:,::-1], index=probs.index, columns=['prob1', 'prob2', 'prob3'])
  probs_sorted['avg'] = 'nan'
  col_avg = []
  for item, row in probs_sorted.iterrows():
    if ( ( row['prob1'] < 0.5 )  & ( row['prob2'] < 0.5) & ( row['prob3'] < 0.5) ):
      avg = (row['prob2'] + row['prob3'])/2
    elif ( ( row['prob1'] > 0.5 )  & ( row['prob2'] > 0.5) ):
      avg = (row['prob1'] + row['prob2'])/2
    elif ( ( row['prob1'] > 0.5 )  & ( row['prob2'] < 0.5) ):
      avg = (row['prob2'] + row['prob3'])/2
    col_avg.append(avg)
  probs_sorted['avg'] = col_avg
  # probs_sorted['avg'] = probs_sorted[['prob1', 'prob2']].mean(axis=1)
  AUC4 = metrics.roc_auc_score(y_t, probs_sorted['avg'])
  print('Fold %.f, Ensemble4 AUC: %.5F' %(key+1, AUC4))
  AUC4s.append(AUC4)

  print('\n')

print(random_state)
print('CV10 AVG AUCs_CTB %.5F, STD: %.5F' %(np.mean(AUCs_CTB), np.std(AUCs_CTB)))
print('CV10 AVG AUCs_LGBM %.5F, STD: %.5F' %(np.mean(AUCs_LGBM), np.std(AUCs_LGBM)))
print('CV10 AVG AUCs_XGB %.5F, STD: %.5F' %(np.mean(AUCs_XGB), np.std(AUCs_XGB)))
print('CV10 Ensemble1 AUC %.5F, STD: %.5F' %(np.mean(AUC1s), np.std(AUC1s)))
print('CV10 Ensemble2 AUC %.5F, STD: %.5F' %(np.mean(AUC2s), np.std(AUC2s)))
print('CV10 Ensemble3 AUC %.5F, STD: %.5F' %(np.mean(AUC3s), np.std(AUC3s)))
print('CV10 Ensemble4 AUC %.5F, STD: %.5F' %(np.mean(AUC4s), np.std(AUC4s)))

## Box Plots
model_names = ['CTB', 'LGBM', 'XGB', 'Ensemble1', 'Ensemble2', 'Ensemble3', 'Ensemble4']
fig, ((ax1)) = plt.subplots(1, 1, figsize=(8, 6))
ax1.boxplot([AUCs_CTB, AUCs_LGBM, AUCs_XGB, AUC1s, AUC2s, AUC3s, AUC4s] )
ax1.set_title('Boxplot of AUC values - CV10 - ' + year , size=16) # Title
ax1.set_ylabel('AUC', fontsize = 14) # Y label
ax1.set_xlabel('Model', fontsize = 14) # X label
ax1.set_xticklabels(model_names, rotation=45, fontsize=14)
plt.show()
print('\n')

# os.chdir('/content/drive/MyDrive/Research/BPY/Evaluation_Results/all')

# fig.savefig(year + ' CV10' + '.png', bbox_inches='tight')

# df_AUCs_CTB = pd.DataFrame(AUCs_CTB, columns=['AUC'])
# df_AUCs_CTB.to_csv(year + ' CV10 ' +'AUCs_CTB.csv')

# df_AUCs_LGBM = pd.DataFrame(AUCs_LGBM, columns=['AUC'])
# df_AUCs_LGBM.to_csv(year + ' CV10 ' +'AUCs_LGBM.csv')

# df_AUCs_XGB = pd.DataFrame(AUCs_XGB, columns=['AUC'])
# df_AUCs_XGB.to_csv(year + ' CV10 ' +'AUCs_XGB.csv')

# Ensemble1 = pd.DataFrame(AUC1s, columns=['AUC'])
# Ensemble1.to_csv(year + ' CV10 ' +'Ensemble1.csv')

# Ensemble2 = pd.DataFrame(AUC2s, columns=['AUC'])
# Ensemble2.to_csv(year + ' CV10 ' +'Ensemble2.csv')

# Ensemble3= pd.DataFrame(AUC3s, columns=['AUC'])
# Ensemble3.to_csv(year + ' CV10 ' +'Ensemble3.csv')

# Ensemble4= pd.DataFrame(AUC4s, columns=['AUC'])
# Ensemble4.to_csv(year + ' CV10 ' +'Ensemble4.csv')

avg_AUCs = []
# avg_AUCs.append(['CTB', np.mean(AUCs_CTB), np.std(AUCs_CTB)])
# avg_AUCs.append(['LGBM', np.mean(AUCs_LGBM), np.std(AUCs_LGBM)])
# avg_AUCs.append(['XGB', np.mean(AUCs_XGB), np.std(AUCs_XGB)])
# avg_AUCs.append(['Ensemble1', np.mean(AUC1s), np.std(AUC1s)])
# avg_AUCs.append(['Ensemble2', np.mean(AUC2s), np.std(AUC2s)])
# avg_AUCs.append(['Ensemble3', np.mean(AUC3s), np.std(AUC3s)])
# avg_AUCs.append(['Ensemble4', np.mean(AUC4s), np.std(AUC4s)])
# avg_AUCs
df_avg_AUC = pd.DataFrame(avg_AUCs, columns=['Model', 'avg_AUC', 'STD_AUC'])
# df_avg_AUC.to_csv('_' + year + ' CV10 ' +'AVG AUCs.csv')
df_avg_AUC

# Tests

In [ ]:
# from scipy import stats
# year='year1'
# a=pd.read_csv(year + ' CV  AUCs_' + 'w2_voting' + '.csv')['AUC']
# b=pd.read_csv(year + ' CV  AUCs_' + 'CBT' + '.csv')['AUC']

# stats.ttest_ind(a, b).pvalue

In [ ]:

Over_sample_method = 'SMOTE'
ratio = 1


In [ ]:
from scipy.stats import wilcoxon

os.chdir('/content/drive/MyDrive/Research/BPY/Evaluation_Results/all')
years = ['year1','year2','year3','year4','year5']
model_names1 = ['CTB', 'LGBM', 'XGB', 'Ensemble1', 'Ensemble2', 'Ensemble3', 'Ensemble4']
model_names2 = ['CTB', 'LGBM', 'XGB', 'Ensemble1', 'Ensemble2', 'Ensemble3', 'Ensemble4']

res = []
for model_name1 in model_names1:
  for model_name2 in model_names2:
    if model_name1 != model_name2:
      res1 = []
      res1.append(model_name1)
      res1.append(model_name2)
      for year in years:
        try:
          a=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_CV10_AUCs_' + model_name1 + '.csv')['AUC']
        except:
          a=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_CV10_' + model_name1 + '.csv')['AUC']
        try:
          b=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_CV10_AUCs_' + model_name2 + '.csv')['AUC']
        except:
          b=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_CV10_' + model_name2 + '.csv')['AUC']
        res1.append(wilcoxon(a-b, alternative='greater').pvalue)
      res.append(res1)
    else:
      pass

res_df_CV = pd.DataFrame(res, columns=['Model1', 'Model2', 'Year1', 'Year2', 'Year3', 'Year4', 'Year5'])

os.chdir('/content/drive/MyDrive/Research/BPY/Evaluation_Results')
res_df_CV.to_csv('wilcoxon_pvalues_cv10_' + str(ratio) + '.csv')
res_df_CV

/usr/local/lib/python3.8/dist-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/usr/local/lib/python3.8/dist-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,Model1,Model2,Year1,Year2,Year3,Year4,Year5
0,CTB,LGBM,0.993164,0.004883,0.032227,0.024414,0.187500
1,CTB,XGB,0.784180,0.024414,0.013672,0.096680,0.246094
2,CTB,Ensemble1,0.998047,0.161133,0.862305,0.539062,0.812500
3,CTB,Ensemble2,1.000000,0.975586,0.995117,0.919922,0.975586
4,CTB,Ensemble3,0.998047,0.687500,0.975586,0.687500,0.903320
5,CTB,Ensemble4,1.000000,0.967773,0.975586,0.838867,0.919922
6,LGBM,CTB,0.009766,0.997070,0.975586,0.981445,0.838867
7,LGBM,XGB,0.009766,0.975586,0.137695,0.903320,0.500000
8,LGBM,Ensemble1,0.422852,0.999023,1.000000,1.000000,0.919922
9,LGBM,Ensemble2,0.958008,1.000000,1.000000,1.000000,0.975586


In [ ]:
from scipy.stats import wilcoxon

os.chdir('/content/drive/MyDrive/Research/BPY/Evaluation_Results/all')
years = ['year1','year2','year3','year4','year5']
model_names1 = ['CTB', 'LGBM', 'XGB', 'Ensemble1', 'Ensemble2', 'Ensemble3', 'Ensemble4']
model_names2 = ['CTB', 'LGBM', 'XGB', 'Ensemble1', 'Ensemble2', 'Ensemble3', 'Ensemble4']
# Ensemble2_AUCs.to_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_hold-out' + '_Ensemble2.csv')
res = []
for model_name1 in model_names1:
  for model_name2 in model_names2:
    if model_name1 != model_name2:
      res1 = []
      res1.append(model_name1)
      res1.append(model_name2)
      for year in years:
        try:
          a=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_hold-out_AUCs_' + model_name1 + '.csv')['AUC']
        except:
          a=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_hold-out_' + model_name1 + '.csv')['AUC']
        try:
          b=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_hold-out_AUCs_' + model_name2 + '.csv')['AUC']
        except:
          b=pd.read_csv(year + '_' + Over_sample_method + '_' + str(ratio) + '_hold-out_' + model_name2 + '.csv')['AUC']
        res1.append(wilcoxon(a-b, alternative='greater').pvalue)
      res.append(res1)
    else:
      pass

res_df_CV = pd.DataFrame(res, columns=['Model1', 'Model2', 'Year1', 'Year2', 'Year3', 'Year4', 'Year5'])

os.chdir('/content/drive/MyDrive/Research/BPY/Evaluation_Results')
res_df_CV.to_csv('wilcoxon_pvalues_hold-out10_' + str(ratio) + '.csv')
res_df_CV

/usr/local/lib/python3.8/dist-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/usr/local/lib/python3.8/dist-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,Model1,Model2,Year1,Year2,Year3,Year4,Year5
0,CTB,LGBM,0.934570,0.246094,0.278320,0.080078,0.422852
1,CTB,XGB,0.862305,0.024414,0.065430,0.187500,0.476383
2,CTB,Ensemble1,0.975586,0.539062,0.934570,0.838867,0.862305
3,CTB,Ensemble2,0.967773,0.687500,0.986328,0.986328,0.862305
4,CTB,Ensemble3,0.975586,0.577148,0.981445,0.919922,0.862305
5,CTB,Ensemble4,0.967773,0.883789,0.993164,0.993164,0.903320
6,LGBM,CTB,0.080078,0.784180,0.753906,0.934570,0.615234
7,LGBM,XGB,0.347656,0.137695,0.460938,0.975586,0.500000
8,LGBM,Ensemble1,0.958008,0.995117,0.986328,1.000000,0.967773
9,LGBM,Ensemble2,0.986328,0.998047,0.986328,1.000000,0.958008
